<a href="https://colab.research.google.com/github/mhdxshahiq/FInal-year-project/blob/main/finetunellm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install unsloth

In [2]:
from unsloth import FastLanguageModel
import torch

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [3]:
max_seq_length = 1024   # Unsloth support RoPE scalling internally
dtype = None            # auto detection float16
load_in_4bit = True     # use 4bit quantization to reduce memry usage

In [4]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/mistral-7b-instruct-v0.2-bnb-4bit",  # example 3B
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit
)

==((====))==  Unsloth 2026.2.1: Fast Mistral patching. Transformers: 4.57.6.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.563 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.10.0+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.6.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.35. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [5]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = [
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = True,
    random_state= 3407
)



Unsloth 2026.2.1 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [6]:
model.print_trainable_parameters()

trainable params: 41,943,040 || all params: 7,283,675,136 || trainable%: 0.5758


In [7]:
from datasets import Dataset

train_data = [
{
"text": """You are a Tesla Model Y assistant.
Answer ONLY using provided context.
If answer not found, say: "The information is not available in the manual."

### Question:
How do I charge the car?

### Answer:
The information is not available in the manual."""
}
]

dataset = Dataset.from_list(train_data)

In [8]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=1024,
    args=TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        num_train_epochs=1,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=10,
        output_dir="outputs",
        optim="adamw_8bit",
    ),
)

trainer.train()

num_proc must be <= 1. Reducing num_proc to 1 for dataset of size 1.


Unsloth: Tokenizing ["text"] (num_proc=1):   0%|          | 0/1 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1 | Num Epochs = 1 | Total steps = 1
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 4 x 1) = 4
 "-____-"     Trainable parameters = 41,943,040 of 7,283,675,136 (0.58% trained)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 3


wandb: You chose "Don't visualize my results"
wandb: Using W&B in offline mode.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


wandb: Detected [huggingface_hub.inference, openai] in use.
wandb: Use W&B Weave for improved LLM call tracing. Install Weave with `pip install weave` then add `import weave` to the top of your script.
wandb: For more information, check out the docs at: https://weave-docs.wandb.ai/


Step,Training Loss


wandb: WARNING URL not available in offline run


train/epoch,▁
train/global_step,▁
total_flos,2789515223040.0
train/epoch,1
train/global_step,1
train_loss,3.47558
train_runtime,102.4159
train_samples_per_second,0.01
train_steps_per_second,0.01


TrainOutput(global_step=1, training_loss=3.4755825996398926, metrics={'train_runtime': 102.4159, 'train_samples_per_second': 0.01, 'train_steps_per_second': 0.01, 'total_flos': 2789515223040.0, 'train_loss': 3.4755825996398926, 'epoch': 1.0})

In [9]:
model.save_pretrained("tesla_lora")
tokenizer.save_pretrained("tesla_lora")

('tesla_lora/tokenizer_config.json',
 'tesla_lora/special_tokens_map.json',
 'tesla_lora/chat_template.jinja',
 'tesla_lora/tokenizer.model',
 'tesla_lora/added_tokens.json',
 'tesla_lora/tokenizer.json')

In [10]:
!pip install transformers datasets accelerate bitsandbytes trl
!pip install langchain faiss-cpu sentence-transformers pypdf

In [11]:
!pip install langchain langchain-community
!pip install faiss-cpu
!pip install sentence-transformers
!pip install pypdf

In [13]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("/content/tesla model y.pdf")
documents = loader.load()

In [16]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)

docs = splitter.split_documents(documents)

In [19]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

vectorstore = FAISS.from_documents(docs, embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k":3})

/tmp/ipython-input-17207/1748668334.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [35]:
def ask_tesla(question):
    # 🔹 Retrieve relevant documents
    retrieved_docs = retriever.invoke(question)

    # 🔴 If nothing relevant found → return info not available
    if not retrieved_docs:
        return "The information is not available in the manual."

    # Combine retrieved context
    context = "\n\n".join([doc.page_content for doc in retrieved_docs])

    # Build prompt
    prompt = f"""
You are a Tesla Model Y manual assistant.

STRICT RULES:
- Answer ONLY using the provided context.
- Only answer questions about Tesla Model Y.
- If answer is not found in context, say:
"The information is not available in the manual."
- Give the answer in clear numbered points.
- Keep it concise.

Context:
{context}

Question: {question}

Answer:
"""

    # Tokenize and generate
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(
        **inputs,
        max_new_tokens=200,
        temperature=0.2,
        do_sample=False,
        eos_token_id=tokenizer.eos_token_id,
    )

    # Decode the generated answer
    answer = tokenizer.decode(
        outputs[0][inputs["input_ids"].shape[-1]:],
        skip_special_tokens=True
    )

    # Return clean answer
    return answer.strip()

In [37]:
ask_tesla("How do I open the charge port?")

'1. Press and release the button on the Tesla charge cable.\n2. Use the touchscreen:\n   a. Touch Controls and then the Charge Port icon (lightning bolt).\n   b. Navigate to Controls > Charging > Open Charge Port.\n3. Press the bottom of the charge port door when Model Y is unlocked and an authenticated phone is nearby.\n4. Press and hold the button on the charging handle for older Tesla charging equipment.\n5. Pull the rear left door handle for vehicles equipped with a Combined Charging System (CCS) charge port.\n6. Open the rear trunk and pull the charge port release cable downwards to unlatch the charge cable.'